In [ ]:
pip install pandas_ta

In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
#Pandas_ta for adding the technical analysis indicators

#Loading the data file
df = pd.read_csv('yahoo_stock.csv')
df.head(10)

In [ ]:
#Adding technical indicators
df['RSI'] = ta.rsi(df.Close, length=15)
#Fast moving average
df['EMAF'] = ta.ema(df.Close, length=20)
#Medium moving average
df['EMAM'] = ta.ema(df.Close, length=100)
#Slow moving average
df['EMAS'] = ta.ema(df.Close, length=150)

In [ ]:
#Adding target column
#Calculating the difference between the close price and open price
df['Target'] = df['Adj Close'] - df.Open
df['Target'] = df['Target'].shift(-1)


#This is the classification approach whether the price is going up or down 
df['TargetClass'] = [1 if df.Target[i] > 0 else 0 for i in range(len(df))]

#Closing price of the next day
#Obtained by shifting the day 1 back
df['TargetNextClose'] = df['Adj Close'].shift(-1)

In [ ]:
#Removing missing values and unnecessary columns
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

In [ ]:
#Remove index columns as they are not needed
df.drop(['index'], axis=1, inplace=True)

In [ ]:
data_set = df.iloc[:, 0:11]
pd.set_option('display.max_columns', None)

data_set.head(10)

In [ ]:
#Applying the MinMaxScaler so I can have data ranging betwen 0 and 1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0, 1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

In [ ]:
#Multiple feature from data provided to the model
X = []

#This is the number of days you want to look back
backcandles = 15
print(data_set_scaled.shape[0])
#This is the part where the columns are getting processed 
for j in range(8):
  X.append([])
  for i in range(backcandles, data_set_scaled.shape[0]):
    X[j].append(data_set_scaled[i-backcandles:i, j])

#Move axis from 0 to possition 2
X = np.moveaxis(X, [0], [2])

#Choose -2 for last column, classification else -1...
X, yi = np.array(X), np.array(data_set_scaled[backcandles:, -1])
y = np.reshape(yi, (len(yi), 1))

print(X.shape)
print(X)
print(y.shape)
print(y)

In [ ]:
#Splitting the data
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

In [ ]:
import keras
import tensorflow as tf
from keras import optimizers
from keras.layers import LSTM
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import TimeDistributed
from keras.layers import Input, Activation, concatenate
from prompt_toolkit.shortcuts.dialogs import input_dialog

#The model is fed two dimentional matix (days back to predict + the columns )
lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(500, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('sigmoid', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train, y=y_train, batch_size=25, epochs=100, shuffle=True, validation_split = 0.1)

In [ ]:
y_pred = model.predict(X_test)
for i in range(10):
  print(y_pred[i], y_test[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test, color = 'black', label = 'Test')
plt.plot(y_pred, color = 'green', label = 'pred')
plt.legend()
plt.show()

In [ ]:
#References to https://www.youtube.com/watch?v=hpfQE0bTeA4&t=12s
#For the tutorial of how to build the model